# 单摆精确测量虚拟平台

本notebook演示了单摆虚拟平台的主要功能，包括：
1. 单摆基本模拟
2. 理论与实验数据对比
3. 重力加速度测量
4. 交互式单摆模拟

In [ ]:
# 导入所需库
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 从我们的模块导入类
from pendulum_simulation import PendulumSimulation
from data_analyzer import DataAnalyzer

## 1. 基本单摆模拟

首先，我们创建一个基本的单摆模拟，并查看其运动特性。

In [ ]:
# 创建单摆实例
pendulum = PendulumSimulation(
    length=1.0,          # 摆长 (m)
    mass=0.1,            # 质量 (kg)
    gravity=9.8,         # 重力加速度 (m/s²)
    damping=0.1,         # 阻尼系数
    initial_angle=np.pi/6  # 初始角度 (30度)
)

# 运行模拟
print("运行单摆模拟...")
results = pendulum.simulate(t_span=(0, 20), t_points=1000)

# 计算周期
periods, avg_period = pendulum.calculate_periods()
print(f"测量的平均周期: {avg_period:.6f} s")

# 计算重力加速度
g = pendulum.calculate_gravity()
print(f"根据周期计算的重力加速度: {g:.6f} m/s²")

# 可视化结果
fig = pendulum.visualize()
plt.show()

### 单摆动画

下面我们创建单摆运动的动画，直观展示其运动过程。

In [ ]:
# 创建动画
anim = pendulum.animate_pendulum(interval=50)

# 显示动画
from IPython.display import HTML
HTML(anim.to_jshtml())

## 2. 理论与实验数据对比

下面我们创建一个理想的理论模型，以及一个带有阻尼和噪声的"实验"模型，并比较两者的差异。

In [ ]:
# 创建无阻尼理论模型
theory = PendulumSimulation(
    length=1.0,
    mass=0.1,
    gravity=9.8,
    damping=0.01,  # 极小阻尼
    initial_angle=np.pi/6
)

# 运行理论模拟
print("运行理论模型...")
theory_results = theory.simulate(t_span=(0, 10), t_points=500)

# 创建带阻尼和噪声的"实验"模型
experiment = PendulumSimulation(
    length=1.0,
    mass=0.1,
    gravity=9.81,  # 略微不同的重力加速度
    damping=0.15,  # 更大的阻尼
    initial_angle=np.pi/6 + 0.02  # 初始角度有小误差
)

# 运行实验模拟
print("运行\"实验\"模型...")
exp_results = experiment.simulate(t_span=(0, 10), t_points=500)

# 添加测量噪声
print("添加测量噪声...")
np.random.seed(42)  # 设置随机种子以便结果可重现
noise_level = 0.005  # 噪声水平

for key in ['angle', 'x_position', 'y_position']:
    noise = np.random.normal(0, noise_level, len(exp_results[key]))
    exp_results[key] = exp_results[key] + noise

# 重新计算派生数据
exp_results['angular_velocity'] = np.gradient(exp_results['angle'], exp_results['time'])

# 创建数据分析器
analyzer = DataAnalyzer()
analyzer.add_theoretical_data(theory_results)
analyzer.add_experimental_data(exp_results)

# 计算误差指标
print("计算误差指标...")
error_metrics = analyzer.calculate_error_metrics()
print("误差指标:")
for key, value in error_metrics.items():
    print(f"  {key}: {value:.6f}")

# 可视化比较
fig = analyzer.visualize_comparison()
plt.show()

# 导出数据到CSV
analyzer.export_data_to_csv('pendulum_data_comparison.csv')

## 3. 重力加速度测量实验

在这个部分中，我们通过测量不同摆长下的周期来估算重力加速度，这是经典的物理实验方法。

In [ ]:
# 准备不同的摆长
lengths = np.linspace(0.5, 2.0, 10)  # 10个不同长度的摆

# 创建数据分析器
analyzer = DataAnalyzer()

# 运行摆长周期实验
print("运行不同摆长的周期测量...")
results, fig = analyzer.pendulum_length_vs_period_experiment(
    PendulumSimulation,
    lengths,
    gravity=9.8,
    mass=0.1,
    damping=0.05,
    initial_angle=np.pi/12,
    t_span=(0, 20),
    t_points=1000
)

# 显示结果
print(f"\n实验测量的重力加速度: {results['gravity_estimate']['g_value']:.6f} ± {results['gravity_estimate']['g_uncertainty']:.6f} m/s²")
print(f"相对于标准值 (9.80665 m/s²) 的误差: {abs(results['gravity_estimate']['g_value'] - 9.80665)/9.80665*100:.6f}%")

# 显示图表
plt.show()

## 4. 交互式单摆模拟

最后，我们创建一个交互式界面，可以实时调整单摆参数并观察其影响。

In [ ]:
import ipywidgets as widgets
from IPython.display import display

# 参数范围
length_slider = widgets.FloatSlider(value=1.0, min=0.1, max=2.0, step=0.1, description='摆长 (m):')
mass_slider = widgets.FloatSlider(value=0.1, min=0.01, max=1.0, step=0.01, description='质量 (kg):')
gravity_slider = widgets.FloatSlider(value=9.8, min=1.0, max=20.0, step=0.1, description='重力加速度:')
damping_slider = widgets.FloatSlider(value=0.1, min=0.0, max=1.0, step=0.01, description='阻尼系数:')
angle_slider = widgets.FloatSlider(value=30, min=5, max=90, step=5, description='初始角度 (°):')

# 输出控件
output = widgets.Output()

# 运行按钮
run_button = widgets.Button(description="运行模拟")

def on_run_button_click(b):
    with output:
        output.clear_output()
        
        # 创建单摆
        pendulum = PendulumSimulation(
            length=length_slider.value,
            mass=mass_slider.value,
            gravity=gravity_slider.value,
            damping=damping_slider.value,
            initial_angle=np.deg2rad(angle_slider.value)  # 转换为弧度
        )
        
        # 运行模拟
        print("运行模拟中...")
        results = pendulum.simulate(t_span=(0, 20), t_points=1000)
        
        # 计算周期
        periods, avg_period = pendulum.calculate_periods()
        print(f"测量的平均周期: {avg_period:.6f} s")
        
        # 计算重力加速度
        g = pendulum.calculate_gravity()
        print(f"根据周期计算的重力加速度: {g:.6f} m/s²")
        
        # 理论周期 (小角近似)
        theory_period = 2 * np.pi * np.sqrt(pendulum.length / pendulum.gravity)
        print(f"理论周期 (小角近似): {theory_period:.6f} s")
        print(f"相对误差: {abs(avg_period - theory_period)/theory_period*100:.4f}%")
        
        # 显示图表
        fig = pendulum.visualize()
        plt.show()
        
# 连接按钮点击事件
run_button.on_click(on_run_button_click)

# 创建UI布局
print("=== 单摆虚拟平台交互式模式 ===")
print("调整参数并点击'运行模拟'按钮")

ui = widgets.VBox([
    widgets.HBox([length_slider, mass_slider]),
    widgets.HBox([gravity_slider, damping_slider]),
    angle_slider,
    run_button,
    output
])

display(ui)